In [2]:
import time
# import tqdm
from tqdm.notebook import tqdm  # Import the notebook version of tqdm

from datasets import load_dataset
import pandas as pd
import numpy as np
import huggingface_hub
from huggingface_hub import HfFileSystem
hffs = HfFileSystem()
from concurrent.futures import ThreadPoolExecutor, as_completed

import transformers
transformers.logging.set_verbosity_error()
from transformers import AutoTokenizer, AutoModel


/Users/enjalot/code/fineweb-modal/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_parquet("./original-data-00000-of-00099.parquet")

FileNotFoundError: [Errno 2] No such file or directory: './original-data-00000-of-00099.parquet'

In [3]:
df.head()

,text,id,dump,url,file_path,language,language_score,token_count,score,int_score
0,"The Independent Jane\nFor all the love, romanc...",<urn:uuid:0d8a309d-25c5-405d-a08a-c11239f0d717>,CC-MAIN-2013-20,http://austenauthors.net/the-independent-jane,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.974320,845,2.750000,3
1,Taking Play Seriously\nBy ROBIN MARANTZ HENIG\...,<urn:uuid:316c7af5-14e1-4d0b-9576-753e17ef2cc5>,CC-MAIN-2013-20,http://query.nytimes.com/gst/fullpage.html?res...,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.961459,1055,2.562500,3
2,How do you get HIV?\nHIV can be passed on when...,<urn:uuid:a3e140cd-7f25-48c9-a2f0-a7d0b1954e0d>,CC-MAIN-2013-20,http://www.childline.org.uk/Explore/SexRelatio...,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.966757,136,3.125000,3
3,CTComms sends on average 2 million emails mont...,<urn:uuid:c337bcd8-6aa1-4f2d-8c48-b916442ebbee>,CC-MAIN-2013-20,http://www.ctt.org/resource_centre/getting_sta...,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.910602,3479,3.234375,3
4,Hold the salt: UCLA engineers develop revoluti...,<urn:uuid:c0b175bb-65fb-420e-a881-a80b91d00ecd>,CC-MAIN-2013-20,http://www.environment.ucla.edu/water/news/art...,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.924981,1115,2.812500,3


In [9]:
# reading in the fetched file
cdf = pd.read_parquet("./embedded-data-00000-of-00099.parquet")


FileNotFoundError: [Errno 2] No such file or directory: './embedded-data-00000-of-00099.parquet'

In [4]:
cdf.head()

NameError: name 'cdf' is not defined

In [5]:
df[df['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]

NameError: name 'df' is not defined

In [6]:
cdf[cdf['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]

NameError: name 'cdf' is not defined

In [7]:
cdf[cdf['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]["chunk_text"]

NameError: name 'cdf' is not defined

In [8]:
# Get the full text string of the rows and display it
full_text = df[df['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]['text'].values[0]
print(full_text)


NameError: name 'df' is not defined

In [15]:
cfull_text = cdf[cdf['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]["chunk_text"].values[1]

In [16]:
print(cfull_text)

. " hurricane force winds extended 125 miles from katrina's center, compared to 60 miles for camille, he said. also, katrina's eye was 32 miles wide even though a storm of its intensity normally has an eye that is 10 miles wide. scientists know much more about surge today than they did in 1969 when camille hit. they can predict with reasonable accuracy what the surge will be. but the unpredictable nature of storms makes it difficult to say far in advance which areas will be flooded and how much higher the wind - whipped waves on top of the surge will be. for example, hurricane ivan last year was predicted to hit west of mobile, alabama. if it had, it would have sent 12 feet of storm surge into the area and 4 feet over the pensacola area. but just hours before landfall, it shifted and hit east of the city. it spared mobile, but sent a 10 - foot surge over the florida panhandle, said stephen baig, a storm surge expert at the national hurricane center in miami. in the case of katrina, est

In [60]:
tokenizer =  AutoTokenizer.from_pretrained("bert-base-uncased")

/Users/enjalot/code/fineweb-modal/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
tokens = tokenizer.encode(full_text)

In [19]:
len(tokens)

901

In [20]:
# weird chunks are because of the 50 overlap. 901 tokens means one 500 chunk, then start at 450 and get 451 tokens. 
# but next start_index is 900 so we get one more chunk of 1 tokens

In [30]:
###
# ---
# all-miniLM
rdf = pd.read_parquet("./redpajama/data-00000-of-00150.parquet")
embeddings = np.memmap("./redpajama/data-00000-of-00150.npy", 
                      dtype='float32', 
                      mode='r',  # 'r' for read-only mode
                      shape=(rdf.shape[0], 384)) 

In [31]:
rdf.head()

,chunk_index,chunk_text,chunk_tokens,chunk_token_count,doc_id,meta
0,0,[CLS] about awb kids are kids join the cast < ...,"[101, 2055, 22091, 2497, 4268, 2024, 4268, 369...",120,2014-15/0000/en_head.json.gz/0,"{""url"": ""http://0360be4.netsolhost.com/news/ch..."
1,1,and support for people with disabilities. she ...,"[1998, 2490, 2005, 2111, 2007, 13597, 1012, 20...",120,2014-15/0000/en_head.json.gz/0,"{""url"": ""http://0360be4.netsolhost.com/news/ch..."
2,2,she takes in her many accomplishments. john ta...,"[2016, 3138, 1999, 2014, 2116, 17571, 1012, 21...",118,2014-15/0000/en_head.json.gz/0,"{""url"": ""http://0360be4.netsolhost.com/news/ch..."
3,0,[CLS] read more music news taylor swift promis...,"[101, 3191, 2062, 2189, 2739, 4202, 9170, 1065...",120,2014-15/0000/en_head.json.gz/1,"{""url"": ""http://1019ampradio.cbslocal.com/2013..."
4,1,apprenticeship. i want to be around people who...,"[20448, 1012, 1045, 2215, 2000, 2022, 2105, 21...",120,2014-15/0000/en_head.json.gz/1,"{""url"": ""http://1019ampradio.cbslocal.com/2013..."


In [32]:
embeddings.shape

(573863, 384)

In [33]:
embeddings[0]

memmap([ 1.53040804e-03,  2.34369095e-02, -3.52175124e-02,
         3.58389970e-03,  4.02170159e-02,  7.93844163e-02,
         5.91101758e-02, -2.55361479e-02,  2.65857689e-02,
        -1.95560772e-02,  1.70425139e-02, -7.23132938e-02,
         2.93341149e-02, -9.89542995e-03,  8.87451097e-05,
        -3.37535664e-02,  1.34171816e-02, -8.10969546e-02,
        -6.17618486e-02, -3.27315703e-02, -7.81138241e-02,
         8.45220312e-02, -3.90154775e-03,  5.01055419e-02,
        -2.42517460e-02, -1.48051661e-02,  5.54365069e-02,
        -1.40428096e-01, -1.78159177e-02, -9.06540155e-02,
         1.84235927e-02, -2.82154419e-02, -2.70691458e-02,
         2.07299944e-02,  1.66558120e-02,  3.04113571e-02,
         3.35049741e-02,  3.90569121e-02, -1.49708956e-01,
        -3.46927010e-02, -2.98830937e-03, -3.94712351e-02,
        -2.21386943e-02, -3.49136740e-02, -5.43454522e-03,
        -7.19265863e-02, -3.39400140e-03, -1.19325213e-01,
         1.45151392e-02, -3.30354087e-02,  1.23088656e-0

In [34]:
# Normalize the first embedding vector
norm = np.linalg.norm(embeddings[0])
normalized_embedding = embeddings[0] / norm
normalized_embedding


array([ 1.53040816e-03,  2.34369114e-02, -3.52175161e-02,  3.58389993e-03,
        4.02170196e-02,  7.93844238e-02,  5.91101795e-02, -2.55361497e-02,
        2.65857708e-02, -1.95560791e-02,  1.70425158e-02, -7.23133013e-02,
        2.93341167e-02, -9.89543088e-03,  8.87451170e-05, -3.37535702e-02,
        1.34171825e-02, -8.10969621e-02, -6.17618524e-02, -3.27315740e-02,
       -7.81138316e-02,  8.45220387e-02, -3.90154799e-03,  5.01055457e-02,
       -2.42517479e-02, -1.48051670e-02,  5.54365106e-02, -1.40428111e-01,
       -1.78159196e-02, -9.06540230e-02,  1.84235945e-02, -2.82154437e-02,
       -2.70691477e-02,  2.07299963e-02,  1.66558139e-02,  3.04113589e-02,
        3.35049778e-02,  3.90569158e-02, -1.49708971e-01, -3.46927047e-02,
       -2.98830960e-03, -3.94712389e-02, -2.21386962e-02, -3.49136777e-02,
       -5.43454569e-03, -7.19265938e-02, -3.39400163e-03, -1.19325221e-01,
        1.45151401e-02, -3.30354124e-02,  1.23088667e-03, -5.83367757e-02,
        3.67367007e-02,  

In [15]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


/Users/enjalot/code/fineweb-modal/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
embst = model.encode(rdf.iloc[0]['chunk_text'], normalize_embeddings=True)
embst1 = model.encode(rdf.iloc[1]['chunk_text'], normalize_embeddings=True)
embstS0 = model.encode(rdf.iloc[156876]['chunk_text'], normalize_embeddings=True)

In [36]:
rdf["original_position"] = np.arange(len(rdf))

In [37]:
rdf.head()

,chunk_index,chunk_text,chunk_tokens,chunk_token_count,doc_id,meta,original_position
0,0,[CLS] about awb kids are kids join the cast < ...,"[101, 2055, 22091, 2497, 4268, 2024, 4268, 369...",120,2014-15/0000/en_head.json.gz/0,"{""url"": ""http://0360be4.netsolhost.com/news/ch...",0
1,1,and support for people with disabilities. she ...,"[1998, 2490, 2005, 2111, 2007, 13597, 1012, 20...",120,2014-15/0000/en_head.json.gz/0,"{""url"": ""http://0360be4.netsolhost.com/news/ch...",1
2,2,she takes in her many accomplishments. john ta...,"[2016, 3138, 1999, 2014, 2116, 17571, 1012, 21...",118,2014-15/0000/en_head.json.gz/0,"{""url"": ""http://0360be4.netsolhost.com/news/ch...",2
3,0,[CLS] read more music news taylor swift promis...,"[101, 3191, 2062, 2189, 2739, 4202, 9170, 1065...",120,2014-15/0000/en_head.json.gz/1,"{""url"": ""http://1019ampradio.cbslocal.com/2013...",3
4,1,apprenticeship. i want to be around people who...,"[20448, 1012, 1045, 2215, 2000, 2022, 2105, 21...",120,2014-15/0000/en_head.json.gz/1,"{""url"": ""http://1019ampradio.cbslocal.com/2013...",4


In [38]:
sdf = rdf.sort_values(by='chunk_token_count', ascending=True)

In [39]:
# sdf = sdf.reset_index(drop=True)

In [40]:
sdf.head(10)

,chunk_index,chunk_text,chunk_tokens,chunk_token_count,doc_id,meta,original_position
156876,0,Terminated.,"[101, 12527, 1012, 102]",4,2014-15/0000/en_head.json.gz/12924,"{""url"": ""http://www.daybydaycartoon.com/2011/1...",156876
211258,0,New Mexico 2006,"[101, 2047, 3290, 2294, 102]",5,2014-15/0000/en_head.json.gz/17569,"{""url"": ""http://www.census.gov/epcd/nonemploye...",211258
370470,0,Mid-Layers,"[101, 3054, 1011, 9014, 102]",5,2014-15/0000/en_middle.json.gz/10330,"{""url"": ""http://www.peterglenn.com/sale/backpa...",370470
348904,0,Third World View,"[101, 2353, 2088, 3193, 102]",5,2014-15/0000/en_middle.json.gz/8809,"{""url"": ""http://wortfm.org/?attachment_id=1206...",348904
547229,0,Beatrix Miller,"[101, 3786, 17682, 4679, 102]",5,2014-15/0000/en_middle.json.gz/22254,"{""url"": ""http://www.vogue.co.uk/magazine/archi...",547229
148794,0,High Recycled Content,"[101, 2152, 22207, 4180, 102]",5,2014-15/0000/en_head.json.gz/12697,"{""url"": ""http://www.armstrong.com/commceilings...",148794
550429,0,"Cosmetics, 2012","[101, 25381, 1010, 2262, 102]",5,2014-15/0000/en_middle.json.gz/22546,"{""url"": ""https://www.myfdb.com/campaigns/14059...",550429
547131,0,New York Arts,"[101, 2047, 2259, 2840, 102]",5,2014-15/0000/en_middle.json.gz/22242,"{""url"": ""http://www.villagevoice.com/2009-07-2...",547131
470908,0,"Greendale,","[101, 2665, 5634, 1010, 102]",5,2014-15/0000/en_middle.json.gz/16785,"{""url"": ""http://www.citysearch.com/profile/map...",470908
412416,0,Patsy and Hedy,"[101, 25382, 1998, 2002, 5149, 102]",6,2014-15/0000/en_middle.json.gz/12983,"{""url"": ""http://www.comics.org/issue/97625/"", ...",412416


In [41]:
i = 0 
for index, row in sdf.iterrows():
    good = row['original_position']
    bad = sdf.iloc[index]['original_position']
    print(i, bad, good)
    i += 1
    if i > 3:
        break


0 362219 156876
1 569232 211258
2 28230 370470
3 145097 348904


In [42]:
sdf.iloc[211258]['original_position']

569232

In [43]:
rdf.iloc[1]['chunk_text']

'and support for people with disabilities. she described the process of creating the two non - profits she manages – “ kids are kids, ” which provides disability awareness workshops and awb which provides theater arts opportunities for children, youth and young adults with physical disabilities. christine talked about the importance of both in increasing inclusion for people, especially young people, with physical disabilities. the march “ everyone has a story ” segment featured christine, her mother, and her brother. christine ’ s mother read a letter she wrote about christine ’ s life and the pride she takes in her many accomplishments. john tartaglia,'

In [44]:
rdf.loc[1]['chunk_text']

'and support for people with disabilities. she described the process of creating the two non - profits she manages – “ kids are kids, ” which provides disability awareness workshops and awb which provides theater arts opportunities for children, youth and young adults with physical disabilities. christine talked about the importance of both in increasing inclusion for people, especially young people, with physical disabilities. the march “ everyone has a story ” segment featured christine, her mother, and her brother. christine ’ s mother read a letter she wrote about christine ’ s life and the pride she takes in her many accomplishments. john tartaglia,'

In [45]:
sdf.iloc[1]['original_position']

211258

In [171]:
rdf.iloc[156876]

chunk_index                                                          0
chunk_text                                                 Terminated.
chunk_tokens                                   [101, 12527, 1012, 102]
chunk_token_count                                                    4
doc_id                              2014-15/0000/en_head.json.gz/12924
meta                 {"url": "http://www.daybydaycartoon.com/2011/1...
original_position                                               156876
Name: 156876, dtype: object

In [172]:
sdf.head()

,chunk_index,chunk_text,chunk_tokens,chunk_token_count,doc_id,meta,original_position
0,0,Terminated.,"[101, 12527, 1012, 102]",4,2014-15/0000/en_head.json.gz/12924,"{""url"": ""http://www.daybydaycartoon.com/2011/1...",156876
1,0,New Mexico 2006,"[101, 2047, 3290, 2294, 102]",5,2014-15/0000/en_head.json.gz/17569,"{""url"": ""http://www.census.gov/epcd/nonemploye...",211258
2,0,Mid-Layers,"[101, 3054, 1011, 9014, 102]",5,2014-15/0000/en_middle.json.gz/10330,"{""url"": ""http://www.peterglenn.com/sale/backpa...",370470
3,0,Third World View,"[101, 2353, 2088, 3193, 102]",5,2014-15/0000/en_middle.json.gz/8809,"{""url"": ""http://wortfm.org/?attachment_id=1206...",348904
4,0,Beatrix Miller,"[101, 3786, 17682, 4679, 102]",5,2014-15/0000/en_middle.json.gz/22254,"{""url"": ""http://www.vogue.co.uk/magazine/archi...",547229


In [186]:
sdf.index[sdf['original_position'] == 1][0]

390537

In [ ]:
348904

In [174]:
sdf[sdf['original_position'] == 1]

,chunk_index,chunk_text,chunk_tokens,chunk_token_count,doc_id,meta,original_position
390537,1,and support for people with disabilities. she ...,"[1998, 2490, 2005, 2111, 2007, 13597, 1012, 20...",120,2014-15/0000/en_head.json.gz/0,"{""url"": ""http://0360be4.netsolhost.com/news/ch...",1


In [187]:
emb = embeddings[390537]

In [188]:
emb

memmap([-4.52683249e-04, -3.97839787e-04, -9.94734373e-03,
        -6.73225820e-02, -4.85355556e-02, -3.06979474e-02,
         8.24557841e-02,  1.39464196e-02,  1.84417851e-02,
        -2.14626696e-02,  1.38744945e-02,  4.01921570e-02,
         1.13227006e-05,  7.11202696e-02,  4.45940197e-02,
        -1.57517586e-02, -6.52511194e-02,  8.23407024e-02,
        -5.11392718e-03, -4.35295179e-02,  1.48167228e-02,
         9.51578841e-03, -4.47378717e-02, -1.86000224e-02,
         4.88646154e-04,  2.95255566e-03,  7.80826882e-02,
        -3.59053798e-02, -2.74325144e-02, -7.83128515e-02,
        -5.31388074e-02, -1.17843293e-01,  3.78905311e-02,
        -5.68789542e-02, -1.62552390e-02,  1.55935213e-02,
        -2.87703350e-02,  3.77610652e-03, -5.57569079e-02,
        -6.57114461e-02,  4.19759192e-02, -9.03963894e-02,
        -1.89884212e-02,  9.98330638e-02,  3.11870426e-02,
         3.60303500e-04, -2.94464380e-02, -5.83462380e-02,
         5.97272143e-02,  2.70728860e-02, -6.35824427e-0

In [189]:
embst1

array([ 1.84022784e-02,  2.13036556e-02,  4.49987389e-02,  1.21036032e-02,
        3.39384377e-02,  1.13584854e-01,  3.90453674e-02,  2.26736497e-02,
        3.68181951e-02, -5.98730631e-02,  3.80697660e-02,  3.54915159e-03,
        2.54673902e-02, -3.40974666e-02,  4.99734050e-03, -1.17743639e-02,
       -3.49157080e-02, -4.09054160e-02,  4.39816527e-02, -3.41169015e-02,
       -8.62772912e-02,  3.01656220e-02,  2.53191236e-02,  8.39999840e-02,
       -4.86296453e-02,  8.16029981e-02, -3.52350026e-02, -9.84868482e-02,
        4.10993397e-03, -3.54244709e-02, -7.96345994e-03, -6.73536509e-02,
       -1.22820837e-02,  3.58506553e-02,  3.37732211e-02,  5.99772334e-02,
        6.13809228e-02,  3.00174113e-02, -7.72080421e-02,  2.29537189e-02,
       -5.34398519e-02, -6.76123798e-02, -1.48260146e-02, -7.10379630e-02,
       -8.60220566e-03, -6.43919930e-02,  2.76593734e-02, -7.08175600e-02,
       -4.76730801e-02,  2.29088310e-02, -1.88694652e-02, -3.20182666e-02,
        3.41766775e-02, -

In [47]:
np.dot(embst1, emb)

NameError: name 'emb' is not defined

In [140]:
rdf.loc[298865]

chunk_index                                                         29
chunk_text           in fact warrem jeffs is in a federal prison. y...
chunk_tokens         [1999, 2755, 2162, 28578, 5076, 2015, 2003, 19...
chunk_token_count                                                  120
doc_id                             2014-15/0000/en_middle.json.gz/4964
meta                 {"url": "http://world.time.com/2012/09/13/prot...
original_position                                               298865
Name: 298865, dtype: object

In [106]:
sdf.iloc[0]['chunk_text']

'Terminated.'

In [50]:
embst0 = model.encode(rdf.iloc[1]['chunk_text'], normalize_embeddings=True)

In [51]:
rdf.iloc[1]['chunk_text']

'and support for people with disabilities. she described the process of creating the two non - profits she manages – “ kids are kids, ” which provides disability awareness workshops and awb which provides theater arts opportunities for children, youth and young adults with physical disabilities. christine talked about the importance of both in increasing inclusion for people, especially young people, with physical disabilities. the march “ everyone has a story ” segment featured christine, her mother, and her brother. christine ’ s mother read a letter she wrote about christine ’ s life and the pride she takes in her many accomplishments. john tartaglia,'

In [52]:
# Calculate cosine similarity between embst and all embeddings
similarities = np.dot(embeddings, embst0)

# Find index of most similar embedding
most_similar_idx = np.argmax(similarities)

print(f"Most similar embedding index: {most_similar_idx}")
print(f"Similarity score: {similarities[most_similar_idx]}")

# Get the corresponding text
print("\nMost similar text:")
print(sdf.iloc[most_similar_idx]['chunk_text'])


Most similar embedding index: 1
Similarity score: 0.9999977946281433

Most similar text:
New Mexico 2006


In [182]:
rdf.iloc[348904]['chunk_text']

'Third World View'

In [54]:
embst0

array([ 1.84022784e-02,  2.13036556e-02,  4.49987389e-02,  1.21036032e-02,
        3.39384377e-02,  1.13584854e-01,  3.90453674e-02,  2.26736497e-02,
        3.68181951e-02, -5.98730631e-02,  3.80697660e-02,  3.54915159e-03,
        2.54673902e-02, -3.40974666e-02,  4.99734050e-03, -1.17743639e-02,
       -3.49157080e-02, -4.09054160e-02,  4.39816527e-02, -3.41169015e-02,
       -8.62772912e-02,  3.01656220e-02,  2.53191236e-02,  8.39999840e-02,
       -4.86296453e-02,  8.16029981e-02, -3.52350026e-02, -9.84868482e-02,
        4.10993397e-03, -3.54244709e-02, -7.96345994e-03, -6.73536509e-02,
       -1.22820837e-02,  3.58506553e-02,  3.37732211e-02,  5.99772334e-02,
        6.13809228e-02,  3.00174113e-02, -7.72080421e-02,  2.29537189e-02,
       -5.34398519e-02, -6.76123798e-02, -1.48260146e-02, -7.10379630e-02,
       -8.60220566e-03, -6.43919930e-02,  2.76593734e-02, -7.08175600e-02,
       -4.76730801e-02,  2.29088310e-02, -1.88694652e-02, -3.20182666e-02,
        3.41766775e-02, -

In [53]:
embeddings[1]

memmap([ 1.83914974e-02,  2.10425239e-02,  4.51277047e-02,
         1.20576527e-02,  3.40717174e-02,  1.13391645e-01,
         3.92532721e-02,  2.27295421e-02,  3.70239988e-02,
        -6.00397326e-02,  3.79880071e-02,  3.71670933e-03,
         2.55311951e-02, -3.40717174e-02,  4.95184679e-03,
        -1.19371517e-02, -3.49453539e-02, -4.09402885e-02,
         4.40733209e-02, -3.41319703e-02, -8.63993689e-02,
         3.02307419e-02,  2.54106931e-02,  8.39290991e-02,
        -4.85318638e-02,  8.13985690e-02, -3.51863541e-02,
        -9.84494910e-02,  4.03302489e-03, -3.55177335e-02,
        -8.10370687e-03, -6.73601776e-02, -1.21630915e-02,
         3.58189866e-02,  3.37704644e-02,  5.99192306e-02,
         6.14556186e-02,  2.99144275e-02, -7.73617774e-02,
         2.30006687e-02, -5.35326637e-02, -6.74806833e-02,
        -1.48065872e-02, -7.06739649e-02, -8.72880686e-03,
        -6.42874017e-02,  2.75797155e-02, -7.08547160e-02,
        -4.76582311e-02,  2.30910461e-02, -1.89036280e-0

In [185]:
np.dot(embeddings[156876], embst)

0.9999986

In [58]:
rdf.iloc[1]['chunk_text']

'and support for people with disabilities. she described the process of creating the two non - profits she manages – “ kids are kids, ” which provides disability awareness workshops and awb which provides theater arts opportunities for children, youth and young adults with physical disabilities. christine talked about the importance of both in increasing inclusion for people, especially young people, with physical disabilities. the march “ everyone has a story ” segment featured christine, her mother, and her brother. christine ’ s mother read a letter she wrote about christine ’ s life and the pride she takes in her many accomplishments. john tartaglia,'

In [56]:
rdf.iloc[most_similar_idx]['chunk_text']

'and support for people with disabilities. she described the process of creating the two non - profits she manages – “ kids are kids, ” which provides disability awareness workshops and awb which provides theater arts opportunities for children, youth and young adults with physical disabilities. christine talked about the importance of both in increasing inclusion for people, especially young people, with physical disabilities. the march “ everyone has a story ” segment featured christine, her mother, and her brother. christine ’ s mother read a letter she wrote about christine ’ s life and the pride she takes in her many accomplishments. john tartaglia,'

In [62]:
model2 = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [77]:
# Get the full hidden states for a specific chunk of text
text = rdf.iloc[1]['chunk_text']

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
# Get attention mask from the inputs
attention_mask = inputs['attention_mask']

# Get the model outputs, setting output_hidden_states=True to get all layers
with torch.no_grad():
    outputs = model2(**inputs)#, output_hidden_states=True)

# Get all hidden states - this will be a tuple with one tensor per layer
# hidden_states = outputs.hidden_states

print(f"Number of hidden layers: {len(outputs[0])}")
print(f"Shape of hidden states from last layer: {outputs[0][-1].shape}")

# The hidden states from each layer will have shape:
# (batch_size, sequence_length, hidden_size)


Number of hidden layers: 1
Shape of hidden states from last layer: torch.Size([122, 384])


In [69]:
import torch

In [70]:

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [78]:
emb = mean_pooling(outputs, attention_mask)

In [79]:
# Normalize embeddings to unit length
emb_norm = emb / emb.norm(dim=1, keepdim=True)
emb_norm

tensor([[ 1.8402e-02,  2.1304e-02,  4.4999e-02,  1.2104e-02,  3.3938e-02,
          1.1358e-01,  3.9045e-02,  2.2674e-02,  3.6818e-02, -5.9873e-02,
          3.8070e-02,  3.5491e-03,  2.5467e-02, -3.4097e-02,  4.9973e-03,
         -1.1774e-02, -3.4916e-02, -4.0905e-02,  4.3982e-02, -3.4117e-02,
         -8.6277e-02,  3.0166e-02,  2.5319e-02,  8.4000e-02, -4.8630e-02,
          8.1603e-02, -3.5235e-02, -9.8487e-02,  4.1099e-03, -3.5424e-02,
         -7.9635e-03, -6.7354e-02, -1.2282e-02,  3.5851e-02,  3.3773e-02,
          5.9977e-02,  6.1381e-02,  3.0017e-02, -7.7208e-02,  2.2954e-02,
         -5.3440e-02, -6.7612e-02, -1.4826e-02, -7.1038e-02, -8.6022e-03,
         -6.4392e-02,  2.7659e-02, -7.0817e-02, -4.7673e-02,  2.2909e-02,
         -1.8869e-02, -3.2018e-02,  3.4177e-02, -2.9265e-02,  2.9598e-02,
          6.9458e-02,  5.5180e-02, -7.4424e-02, -4.8928e-03, -8.5835e-02,
         -2.9040e-02, -1.1016e-02, -5.2444e-02,  1.6708e-02,  2.6626e-02,
         -1.8043e-02, -3.5924e-03,  6.

In [75]:
embst0


array([ 1.84022784e-02,  2.13036556e-02,  4.49987389e-02,  1.21036032e-02,
        3.39384377e-02,  1.13584854e-01,  3.90453674e-02,  2.26736497e-02,
        3.68181951e-02, -5.98730631e-02,  3.80697660e-02,  3.54915159e-03,
        2.54673902e-02, -3.40974666e-02,  4.99734050e-03, -1.17743639e-02,
       -3.49157080e-02, -4.09054160e-02,  4.39816527e-02, -3.41169015e-02,
       -8.62772912e-02,  3.01656220e-02,  2.53191236e-02,  8.39999840e-02,
       -4.86296453e-02,  8.16029981e-02, -3.52350026e-02, -9.84868482e-02,
        4.10993397e-03, -3.54244709e-02, -7.96345994e-03, -6.73536509e-02,
       -1.22820837e-02,  3.58506553e-02,  3.37732211e-02,  5.99772334e-02,
        6.13809228e-02,  3.00174113e-02, -7.72080421e-02,  2.29537189e-02,
       -5.34398519e-02, -6.76123798e-02, -1.48260146e-02, -7.10379630e-02,
       -8.60220566e-03, -6.43919930e-02,  2.76593734e-02, -7.08175600e-02,
       -4.76730801e-02,  2.29088310e-02, -1.88694652e-02, -3.20182666e-02,
        3.41766775e-02, -